In [10]:
#this is array and table and machine learning stuff
import math
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#this is xgboost stuff
from xgboost import XGBClassifier
from xgboost import XGBModel
from xgboost import XGBRegressor

#this is catboost
from catboost import CatBoostClassifier, Pool
import ipywidgets
import IPython

#this is plot stuff
import matplotlib.pyplot as plt
import seaborn as sns

#idk what this is but Data100 used it
import scipy.stats

# Gotta Add Our Table First!

apples

In [11]:
url = 'apple_quality.csv'
apple = pd.read_csv(url)

In [12]:
apple

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good
...,...,...,...,...,...,...,...,...,...
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806,good
3999,3999.0,0.278540,-1.715505,0.121217,-1.154075,1.266677,-0.776571,1.599796456,good


Next, we need to clean our data to use it in a model. We remove all rows with NaN values. This only removes the last row, but for other sets of data, it might be a bit more.

Afterwards, since we're going to attempt to predict whether an apple is good or bad, we put all variables in the X and the good and bad are our Y. For models, categorical data can be put into binary form, so our models can use numerical values instead. Since we only have 2 outcomes (good or bad), we can just use 0 and 1. 

In [13]:
#cleaned the data to ensure it can be used in a model.
apple = apple.dropna()
apple = apple.replace({"good":1, "bad":0})
X = apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight"]]
Y = apple['Quality']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=42)

/var/folders/78/wrm63b0n5jxg2yyw_6wf_66h0000gn/T/ipykernel_1171/3367397462.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  apple = apple.replace({"good":1, "bad":0})


Let's try using XGB first!

In [14]:
model = XGBClassifier()
model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
predicted_values = model.predict(X_test)
accuracy = accuracy_score(Y_test, predicted_values)
accuracy

0.8875

In [16]:
print("My accuracy is", accuracy, ".")
xgb_percent_accuracy = 100*accuracy
print("My accuracy is", xgb_percent_accuracy, "%!")

My accuracy is 0.8875 .
My accuracy is 88.75 %!


What if we use other models? Let's try logistic regression.

In [17]:
model = lm.LogisticRegression()
model.fit(X_train, Y_train)
log_accuracy = model.score(X_test, Y_test)
log_accuracy

0.7575

In [18]:
print("The logistic model has an accuracy of 75.75%, while the XGBoost model had an accuracy of 88.75%!")

The logistic model has an accuracy of 75.75%, while the XGBoost model had an accuracy of 88.75%!


When I tried to use Acidity in the XBG model earlier, the model said it was the wrong type of data, so I disregarded it. Let's see if using Acidity increases our accuracy!

In [19]:
apple = apple.dropna()
apple = apple.replace({"good":1, "bad":0})
X = apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight", "Acidity"]]
Y = apple['Quality']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=42)

In [20]:
acid_model = lm.LogisticRegression()
acid_model.fit(X_train, Y_train)
log_accuracy_acid = acid_model.score(X_test, Y_test)
log_accuracy_acid

0.75

Adding acidity decreases the accuracy ironically. Are there any other models we could potentially use?

In [21]:
catboost_train_data = Pool(X_train, Y_train)
catboost_model = CatBoostClassifier()

In [22]:
catboost_model.fit(catboost_train_data, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.017802
0:	learn: 0.6821551	total: 51.8ms	remaining: 51.8s
1:	learn: 0.6726690	total: 54.8ms	remaining: 27.4s
2:	learn: 0.6645269	total: 57.2ms	remaining: 19s
3:	learn: 0.6562081	total: 59.6ms	remaining: 14.8s
4:	learn: 0.6476392	total: 62.5ms	remaining: 12.4s
5:	learn: 0.6400178	total: 65.2ms	remaining: 10.8s
6:	learn: 0.6319314	total: 67.6ms	remaining: 9.59s
7:	learn: 0.6239141	total: 70.2ms	remaining: 8.71s
8:	learn: 0.6160814	total: 72.6ms	remaining: 7.99s
9:	learn: 0.6085779	total: 74.9ms	remaining: 7.42s
10:	learn: 0.6023016	total: 77.6ms	remaining: 6.97s
11:	learn: 0.5975043	total: 80.1ms	remaining: 6.59s


12:	learn: 0.5908434	total: 82.7ms	remaining: 6.28s
13:	learn: 0.5863855	total: 86.1ms	remaining: 6.07s
14:	learn: 0.5815882	total: 89.3ms	remaining: 5.87s
15:	learn: 0.5769304	total: 91.7ms	remaining: 5.64s
16:	learn: 0.5716923	total: 94.9ms	remaining: 5.49s
17:	learn: 0.5657041	total: 97.8ms	remaining: 5.33s
18:	learn: 0.5608243	total: 101ms	remaining: 5.19s
19:	learn: 0.5559195	total: 103ms	remaining: 5.05s
20:	learn: 0.5509891	total: 105ms	remaining: 4.92s
21:	learn: 0.5460725	total: 108ms	remaining: 4.81s
22:	learn: 0.5410297	total: 111ms	remaining: 4.72s
23:	learn: 0.5365416	total: 114ms	remaining: 4.62s
24:	learn: 0.5329181	total: 117ms	remaining: 4.57s
25:	learn: 0.5288438	total: 121ms	remaining: 4.53s
26:	learn: 0.5241808	total: 124ms	remaining: 4.46s
27:	learn: 0.5204383	total: 127ms	remaining: 4.42s
28:	learn: 0.5170526	total: 130ms	remaining: 4.35s
29:	learn: 0.5131556	total: 133ms	remaining: 4.31s
30:	learn: 0.5102391	total: 136ms	remaining: 4.26s
31:	learn: 0.5061545	tota

In [ ]:
catboost_model.score(catboost_train_data)

0.97

In [ ]:
catboost_test_data = Pool(X_test)

In [ ]:
catboost_model.predict(catboost_test_data)

array([1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [ ]:
catboost_score = catboost_model.score(X_test, Y_test)
catboost_score

0.92

Catboost has the highest accuracy!

To summarize...

In [ ]:
catboost_percent = catboost_score*100
xgb_percent_accuracy = 100*accuracy
log_percent = log_accuracy*100
print("Catboost has the highest score of", catboost_percent, "%.")
print("XGB has an accuracy of", xgb_percent_accuracy, "%.")
print("The logistic regression has the lowest accuracy, an accuracy of", log_percent, "%.")

Catboost has the highest score of 92.0 %.
XGB has an accuracy of 88.75 %.
The logistic regression has the lowest accuracy, an accuracy of 75.75 %.
